In [50]:
import cv2 as cv 
import numpy as np
import os
import matplotlib.pyplot as plt
import random
import shutil

%matplotlib inline

# Define the directories
current_dir = os.getcwd()
parent_dir = os.path.dirname(current_dir)

data_dir = os.path.join(parent_dir, 'data', 'raw')
output_dir = os.path.join(parent_dir, 'data', 'processed')

train_dir = os.path.join(output_dir,  'train')
test_dir = os.path.join(output_dir, 'test')
os.makedirs(train_dir, exist_ok=True)
os.makedirs(test_dir, exist_ok=True)

files_list = os.listdir(data_dir)

['20240904_083201.jpg', '20240904_083217.jpg', '20240904_083233.jpg', '20240904_083248.jpg', '20240904_083305.jpg', '20240904_083320.jpg', '20240904_083334.jpg', '20240904_083350.jpg', '20240904_083411.jpg', '20240904_083435.jpg', '20240904_091404.jpg', '20240904_091434.jpg', '20240904_091442.jpg', '20240904_091451.jpg', '20240904_091501.jpg', '20240904_091509.jpg']


Read images and convert RGB, BINARY, DILATED


In [51]:
rgb_images = []
binary_images = []
dilated_images = []
print(files_list[-1])
img = files_list[-1]
# for img in files_list[0:1]:
img_path = os.path.join(data_dir, img)
image = cv.imread(img_path)
image_gray = cv.cvtColor(image, cv.COLOR_BGR2GRAY)
_, binary = cv.threshold(image_gray, 127, 255, cv.THRESH_BINARY_INV)
# kernal = np.ones((5,5), np.uint8)
dilated = cv.dilate(binary, None, iterations=2)
rgb_images.append(image)
binary_images.append(binary)
dilated_images.append(dilated)

20240904_091509.jpg


In [52]:
cv.imwrite('0 - RGB.jpg', rgb_images[0])
cv.imwrite('1 - BINARY.jpg', binary_images[0])
cv.imwrite('2 - DILATED.jpg', dilated_images[0])

True

Contours

In [53]:
image = rgb_images[0].copy()
contours , hierarchy = cv.findContours(dilated_images[0], cv.RETR_TREE, cv.CHAIN_APPROX_SIMPLE)
cv.drawContours(image, contours, -1, (0,255,0), 3)
cv.imwrite('3 - ALL CONTOURS.jpg', image)

True

In [54]:
image = rgb_images[0].copy()
rectangles = []
for contour in contours:
    approx = cv.approxPolyDP(contour, 0.02 * cv.arcLength(contour, True), True)
    if len(approx) == 4:
        if cv.isContourConvex(approx):
            cv.drawContours(image, [approx], 0, (0, 255, 0), 2)
            rectangles.append(approx)

cv.imwrite('4 - RECTANGLES.jpg', image)

True

In [55]:
# Max contour area
max_area = 0
max_contour = None
for contour in rectangles:
    area = cv.contourArea(contour)
    if area > max_area:
        max_area = area
        max_contour = contour

image = rgb_images[0].copy()
cv.drawContours(image, [max_contour], 0, (0, 255, 0), 2)
cv.imwrite('5 - MAX RECTANGLE.jpg', image)

True

Warped

In [56]:
points = max_contour.reshape(4, 2)
rect = np.zeros((4, 2), dtype = "float32")

points_sum = points.sum(axis = 1)
rect[0] = points[np.argmin(points_sum)] #tleft
rect[2] = points[np.argmax(points_sum)] #tright

points_diff = np.diff(points, axis = 1)
rect[1] = points[np.argmin(points_diff)]#bright
rect[3] = points[np.argmax(points_diff)]#bleft

(tl, tr, br, bl) = rect

widthA = np.sqrt(((br[0] - bl[0]) ** 2) + ((br[1] - bl[1]) ** 2))
widthB = np.sqrt(((tr[0] - tl[0]) ** 2) + ((tr[1] - tl[1]) ** 2))

maxWidth = max(int(widthA), int(widthB))

heightA = np.sqrt(((tr[0] - br[0]) ** 2) + ((tr[1] - br[1]) ** 2))
heightB = np.sqrt(((tl[0] - bl[0]) ** 2) + ((tl[1] - bl[1]) ** 2))

maxHeight = max(int(heightA), int(heightB))

destination = np.array([
    [0, 0],
    [maxWidth - 1, 0],
    [maxWidth - 1, maxHeight - 1],
    [0, maxHeight - 1]], dtype = "float32")
M = cv.getPerspectiveTransform(rect, destination)
image = rgb_images[0].copy()
warped = cv.warpPerspective(image, M, (maxWidth, maxHeight))

cv.imwrite('6 - WARPED.jpg', warped)

True

In [57]:
image = warped.copy()

image_gray = cv.cvtColor(image, cv.COLOR_BGR2GRAY)
_, binary = cv.threshold(image_gray, 127, 255, cv.THRESH_BINARY_INV)
dilated = cv.dilate(binary, None, iterations=2)

cv.imwrite('7 - BINARY.jpg', binary)
cv.imwrite('8 - DILATED.jpg', dilated)

True

In [58]:
image = warped.copy()

hor = np.array([np.ones(28)])
ver = np.array(np.ones((28,1)))

hor_eroded = cv.erode(dilated, hor, iterations=15)
hor_dilated = cv.dilate(hor_eroded, hor, iterations=15)

ver_eroded = cv.erode(dilated, ver, iterations=18)
ver_dilated = cv.dilate(ver_eroded, ver, iterations=15)

table = cv.add(hor_dilated, ver_dilated)

cv.imwrite('9 - IMAGE TABLE.jpg', table)

lines = cv.HoughLines(table, 1, (np.pi)/4/180, 2000)
no_lines = cv.subtract(table, binary)

blank = np.zeros_like(table)


def extend_line(x1, y1, x2, y2, factor=1.5):
    dx = x2 - x1
    dy = y2 - y1
    x1_new = int(x1 - factor * dx)
    y1_new = int(y1 - factor * dy)
    x2_new = int(x2 + factor * dx)
    y2_new = int(y2 + factor * dy)
    return x1_new, y1_new, x2_new, y2_new

# Draw the lines on the original image
if lines is not None:
    for rho, theta in lines[:, 0]:
        a = np.cos(theta)
        b = np.sin(theta)
        x0 = a * rho
        y0 = b * rho
        x1 = int(x0 + 1000 * (-b))
        y1 = int(y0 + 1000 * (a))
        x2 = int(x0 - 1000 * (-b))
        y2 = int(y0 - 1000 * (a))
        
        # Extend the line
        x1, y1, x2, y2 = extend_line(x1, y1, x2, y2, factor=50)
        
        cv.line(blank, (x1, y1), (x2, y2), (255, 255, 255), 10)

no_lines = cv.subtract(binary, table)

cv.imwrite('10 - NO LINES.jpg', no_lines)

table = blank
cv.imwrite('9 - TABLE.jpg', table)

True

In [59]:
image = warped.copy()
contours , hierarchy = cv.findContours(table, cv.RETR_TREE, cv.CHAIN_APPROX_SIMPLE)
cv.drawContours(image, contours, -1, (0,255,0), 3)
cv.imwrite('10 - ALL CONTOURS.jpg', image)

True

In [60]:
image = warped.copy()
rectangles = []
for contour in contours:
    approx = cv.approxPolyDP(contour, 0.02 * cv.arcLength(contour, True), True)
    if len(approx) == 4:
        if cv.isContourConvex(approx):
            cv.drawContours(image, [approx], 0, (0, 255, 0), 2)
            rectangles.append(approx)

cv.imwrite('11 - RECTANGLES.jpg', image)

True

In [61]:
areas = []
for contour in rectangles:
    area = cv.contourArea(contour)
    if area > 1000:
        areas.append(area)

areas = np.array(areas)
print('areas:', len(areas))

# Calculate the first quartile (Q1) and the third quartile (Q3)
Q1 = np.percentile(areas, 40)
Q3 = np.percentile(areas, 60)

# Compute the Interquartile Range (IQR)
IQR = Q3 - Q1

# Define the lower and upper bounds
lower_bound = Q1 - 7 * IQR
upper_bound = Q3 + 7 * IQR

filtered_rectangles = []
image = warped.copy()
for contour in rectangles:
    area = cv.contourArea(contour)
    if area > lower_bound and area < upper_bound:
        filtered_rectangles.append(contour)
        cv.drawContours(image, [contour], 0, (0, 255, 0), 2)

cv.imwrite('12 - FILTERED RECTANGLES.jpg', image)

areas: 21


True

In [62]:
filtered_rectangles = np.array(filtered_rectangles)
filtered = filtered_rectangles[::-1]
filtered = filtered.reshape(-1, 4,4, 2)

filtered_rectangles = filtered

In [63]:
target_size = (600, 300)

for row in range(len(filtered_rectangles)):
    new_dir = f'{output_dir}\\{row}\\' 
    os.makedirs(new_dir, exist_ok=True)
    for i , contour in enumerate(filtered_rectangles[row]):
        x, y, w, h = cv.boundingRect(contour)
        crop_amount = 15 
        # Crop the image to the bounding rectangle
        cropped_image = no_lines[y+crop_amount:y+h-crop_amount, x+crop_amount:x+w-crop_amount]
        
        cropped_image = cv.resize(cropped_image, target_size)

        # Save the cropped image to the file system
        filename = os.path.join(new_dir, f'image_{i}.jpg')
        cv.imwrite(filename, cropped_image)

In [64]:
dir = os.listdir(data_dir)
for i, d in enumerate(dir):
    print(i, d)
    if i == 10:
        break

0 20240904_083201.jpg
1 20240904_083217.jpg
2 20240904_083233.jpg
3 20240904_083248.jpg
4 20240904_083305.jpg
5 20240904_083320.jpg
6 20240904_083334.jpg
7 20240904_083350.jpg
8 20240904_083411.jpg
9 20240904_083435.jpg
10 20240904_091404.jpg


dividing the images into test and train

In [65]:
# Iterate through each numbered folder in the processed directory
for folder_name in os.listdir(output_dir):
    folder_path = os.path.join(output_dir, folder_name)
    if os.path.isdir(folder_path) and folder_name.isdigit():
        images = [f for f in os.listdir(folder_path) if f.endswith('.jpg')]
        
        # Ensure there are exactly 4 images in the folder
        if len(images) == 4:
            random.shuffle(images)

            test_image = images[0]
            train_images = images[1:]
            
            # Create respective folders in the test and train directories
            test_folder = os.path.join(test_dir, folder_name)
            train_folder = os.path.join(train_dir, folder_name)
            os.makedirs(test_folder, exist_ok=True)
            os.makedirs(train_folder, exist_ok=True)
            
            # Move the test image
            test_image_src = os.path.join(folder_path, test_image)
            test_image_dst = os.path.join(test_folder, test_image)
            shutil.copy(test_image_src, test_image_dst)
            
            # Move the train images
            for train_image in train_images:
                train_image_src = os.path.join(folder_path, train_image)
                train_image_dst = os.path.join(train_folder, train_image)
                shutil.copy(train_image_src, train_image_dst)
        else:
            print(f"Folder {folder_name} does not contain exactly 4 images.")

print("Data split into train and test sets successfully.")

Data split into train and test sets successfully.


In [66]:
#remove the numbered folders 
for folder_name in os.listdir(output_dir):
    folder_path = os.path.join(output_dir, folder_name)
    if os.path.isdir(folder_path) and folder_name.isdigit():
        shutil.rmtree(folder_path)